In [1]:
import dfBasics
import common
import encoder
import pfAdapt
#import charts


In [3]:
import pandas as pd
from pyspark.sql import functions

In [4]:
def getCountDF(pf,column,hashes):
    dft = pd.DataFrame(columns=[column, 'count'])
    i=0
    for hash in hashes:
        pfall=pf[pf[column] == hash]
        num=len(pfall)
        if num > 1:
            dft.loc[i] = [hash] + [num]
            i=i+1
    return dft
    #return dft.sort_values('count')

def usedcolumns(tb,row):
    col = []
    for column in tb.columns:
        if tb.iloc[row][column] == None :
            col.append(column)
    return col

def diffcolumns(tb):
    col = []
    for column in tb.columns:
        if tb.iloc[0][column] != tb.iloc[1][column] :
            col.append(column)
    return col

def printtt():    
    col = []
    for index, row in tt.iterrows():
        tb = pfall[pfall['CGLOBALMESSAGEID'] == row['CGLOBALMESSAGEID']]
        for bindex, brow in tb.iterrows():
            if pfall.loc[bindex]['CSLADELIVERYTIME'] < 0 :
                #print (str(index) + ' ' + str(bindex))
                col.append(bindex)
    return col


In [5]:
def astype(pfall,selected,newtype):
    for each in selected:
        pfall[each] = pfall[each].astype(newtype)     


# B

In [6]:
selected = [ 'CSTATUS', 'CSERVICE',\
        'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
        'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID']


In [7]:
def encodepfall(pfall,number):
    print('1')
    astype(pfall,['CSTARTTIME','CENDTIME','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME','CINBOUNDSIZE'] ,int) 
    #del(pfall['CSLABILLINGMONTH'])
    pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
    print('1b')
    ac = pd.unique(pfall['CGLOBALMESSAGEID'])
    print('2')
    tt = getCountDF(pfall,'CGLOBALMESSAGEID',ac)
    print('3')
    col = printtt() 
    print('4')
    pfall = pfall.drop(col)
    print('5')
    astype(pfall,selected,str) 
    encoder.encode(pfall,selected)
    
    print('6')
    pfall = pfall.drop_duplicates()
    print('7')
    pfall.to_parquet('/tmp/sla_' + number + '.parquet', engine='fastparquet', compression='GZIP')
    return pfall

# Main

In [8]:
columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
       'CSLABILLINGMONTH', 'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
       'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
       'CMESSAGETAT2', 'CSLADELIVERYTIME']
# withot 'CSLABILLINGMONTH'
def get_columns_2():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE',\
            'CSENDERPROTOCOL', 'CSENDERENDPOINTID',\
           'CINBOUNDSIZE', 'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT',\
           'CMESSAGETAT2', 'CSLADELIVERYTIME']
    return columns
columns = get_columns_2()
#to count messages sent
#columns = [ 'CSTARTTIME', 'CSENDERENDPOINTID']

In [9]:
sparkSession = dfBasics.getSparkSession()

In [10]:
# read timestamps from file
with open("/tmp/slatimestamps.txt", "r") as file:
    lines = file.read().split('\n')

In [ ]:
for line in lines:
    df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/' + line +  '*/*').select(columns).dropDuplicates()
    print('df.toPandas')
    pfall = df.toPandas() 
    print('df.toPandas')
    encodepfall(pfall,line)

df.toPandas
df.toPandas
1
1b
2


In [11]:
line = lines[0]
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/sla_sql_data/' + line +  '*/*').select(columns).dropDuplicates()
print('df.toPandas')
pfall = df.toPandas() 

df.toPandas


In [12]:
print('1')
astype(pfall,['CSTARTTIME','CENDTIME','CSLATAT','CMESSAGETAT2','CSLADELIVERYTIME','CINBOUNDSIZE'] ,int) 
#del(pfall['CSLABILLINGMONTH'])
pfall['CGLOBALMESSAGEID'] = pfall['CGLOBALMESSAGEID'].apply(hash)
print('1b')
ac = pd.unique(pfall['CGLOBALMESSAGEID'])

1
1b


In [29]:
mdcountsall = pfall.groupby(['CGLOBALMESSAGEID','CSTATUS'])['CSTARTTIME'].count()
mdcountsall = mdcountsall[mdcountsall > 1]

In [30]:
#len(pfall), len(ac)
#pfall.head()
#pfall.dtypes

mdcountsall

CGLOBALMESSAGEID      CSTATUS
-7106889482465263091  PENDING    2
-6728710525560770143  SUCCESS    2
-6252343954013336673  PENDING    2
-5991088962523514816  SUCCESS    2
-5376917782570268553  SUCCESS    2
-1813780913549002217  SUCCESS    2
-825101255996487945   SUCCESS    2
 244699708392937803   SUCCESS    2
 2918319669369699741  SUCCESS    2
 3198846195559485937  SUCCESS    2
 5087226829455254053  SUCCESS    2
 6235470665220927147  SUCCESS    2
 6973525114254465644  SUCCESS    2
 8093386665392717581  SUCCESS    2
 8486561839724650877  SUCCESS    2
Name: CSTARTTIME, dtype: int64

In [31]:
pfall[pfall['CGLOBALMESSAGEID'] == mdcountsall.index.get_level_values(0)[0]]
#pfall['CGLOBALMESSAGEID'] == -9209718302575659389


,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERPROTOCOL,CSENDERENDPOINTID,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME
2240965,-7106889482465263091,1575886202504,1575886204603,PENDING,InvoicePortal,AS2,0624a3a0-85da-11e9-b822-0ff9ac1b495c_f77ef920-...,14919,AS2,803ca7e0-198e-11e9-b9de-0e92ac1b495c,0,2099,-1
5774706,-7106889482465263091,1575886202504,1575886202685,PENDING,InvoicePortal,AS2,0624a3a0-85da-11e9-b822-0ff9ac1b495c_f77ef920-...,14919,AS2,803ca7e0-198e-11e9-b9de-0e92ac1b495c,0,181,-1


In [15]:
#len(pfall[pfall['CSLADELIVERYTIME'] == -1])
mdcountsall.index.get_level_values(0)[0]

-9222311199239246750

In [26]:
def printtt(tt):    
    col = []
    i = 0
    for index in tt.index.get_level_values(0):
        tb = pfall[pfall['CGLOBALMESSAGEID'] == index]
        findex = []
        for bindex, brow in tb.iterrows():   
            if pfall.loc[bindex]['CSLADELIVERYTIME'] < 0 :
                #print (str(index) + ' ' + str(bindex))
                #col.append(bindex)
                findex.append(bindex)
        if(len(findex) == 1):
            col.append(findex[0])   
        else:
            if()
    return col

f = printtt(mdcountsall)

In [27]:
len(f), len(mdcountsall)

(1248, 1263)

In [28]:
pfall = pfall.drop(f)